## Toggle Flip-Flop

In this example we create a toggle flip-flop (TFF) from a d-flip-flop and an xor gate. In `Magma`, sequential logic circuits can be constructed by composing combinational logic with register primitives, such as the ice40 DFF.

In [ ]:
import magma as m
m.set_mantle_target("ice40")

from mantle import DFF

import lattice ice40


In [ ]:
def tff():
    # instance a dff to hold the state of the toggle flip-flop - this needs to be done first
    ff = DFF()

    # compute the next state as the not of the old state ff.O
    ff( ~ff.O )

    return ff.O

In [ ]:
from loam.boards.icestick import IceStick

icestick = IceStick()
icestick.Clock.on()
icestick.J3[0].rename('J3').output().on()

main = icestick.DefineMain()
m.wire( tff(), main.J3 )
m.EndDefine()

## Compile and Build

In [ ]:
m.compile("build/tff", main)

In [1]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif tff.blif' tff.v
arachne-pnr -q -d 1k -o tff.txt -p tff.pcf tff.blif 
icepack tff.txt tff.bin
# iceprog tff.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


note: no set_io constraints for pin `J1'
tff.pcf:3: fatal error: Missing 1 set_io constraints


Pin 0 of J3 should be set to the value of Pin 1 of J1.

In [ ]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif dff.blif' dff.v
arachne-pnr -q -d 1k -o dff.txt -p dff.pcf dff.blif 
icepack dff.txt dff.bin
iceprog dff.bin